# Credit
Fork from https://www.kaggle.com/code/rsuhara/ai-generated-text-detection-quick-baseline

Inspired by VLADIMIR DEMIDOV's work : <br>
https://www.kaggle.com/code/yekenot/llm-detect-by-regression

For the training data we shall use the "RDizzl3 seven" dataset (v1) which can be found in the "LLM: 7 prompt training dataset" https://www.kaggle.com/datasets/carlmcbrideellis/llm-7-prompt-training-dataset



# Importing library

In [3]:
import numpy as np
import pandas as pd
import warnings
warnings.filterwarnings('ignore')

In [7]:
final_csv= pd.DataFrame()
csv = pd.read_csv('/kaggle/input/homonyms-problem/Twitter_Data.csv')
csv = csv[csv['category'] != 0]
csv['category'].replace(-1 , 0 ,inplace = True)

csv.reset_index(inplace=True)
# csv['label_index'] = 1
# for index,x in enumerate(csv['Label']):
#     if x == 'negative':
#         csv['label_index'][index] = 0
    
final_csv[['text','label']] = csv[['clean_text','category']]

In [24]:
final_csv.dropna(inplace=True)

In [25]:
# final_csv= pd.DataFrame()
# csv = pd.read_csv('/kaggle/input/homonyms-problem/Twitter_Data.csv')
# csv = csv[csv['Label'] != 'litigious' ]
# csv = csv[csv['Label'] != 'uncertainty' ]
# csv.reset_index(inplace=True)
# csv['label_index'] = 1
# for index,x in enumerate(csv['Label']):
#     if x == 'negative':
#         csv['label_index'][index] = 0
    
# final_csv[['text','label']] = csv[['Text','label_index']]

In [73]:
final_csv['label'] = final_csv['label'].astype('int')
final_csv

,text,label
0,when modi promised “minimum government maximum...,0
1,what did just say vote for modi welcome bjp t...,1
2,asking his supporters prefix chowkidar their n...,1
3,answer who among these the most powerful world...,1
4,with upcoming election india saga going import...,1
...,...,...
107762,engine growth modi unveils indias first 12000 ...,1
107763,modi promised 2014 lok sabha elections that be...,1
107764,why these 456 crores paid neerav modi not reco...,0
107765,dear rss terrorist payal gawar what about modi...,0


In [74]:
import numpy as np
from sklearn.model_selection import train_test_split
train, validation = train_test_split(final_csv, test_size=0.03, random_state=42)
train.reset_index(inplace=True)
validation.reset_index(inplace=True)

# Importing files and Feature Engineering

In [75]:
# external_df = pd.read_csv("/kaggle/input/daigt-external-dataset/daigt_external_dataset.csv", sep=',')
# print(external_df.shape)
# external_df = external_df.rename(columns={'generated': 'label'})
# external_df = external_df[["source_text"]]
# external_df.columns = ["text"]
# external_df['text'] = external_df['text'].str.replace('\n', '')
# external_df["label"] = 1

In [76]:
# train = pd.read_csv("/kaggle/input/llm-7-prompt-training-dataset/train_essays_RDizzl3_seven_v1.csv")
# train=pd.concat([train,external_df])
# test = pd.read_csv('/kaggle/input/llm-detect-ai-generated-text/test_essays.csv')

# df = pd.concat([train['text'], test['text']], axis=0)

# vectorizer = TfidfVectorizer(stop_words='english',ngram_range=(1, 3),sublinear_tf=True)
# X = vectorizer.fit_transform(df)

In [77]:
train = train[['text','label']]
valid = validation[['text','label']]

In [78]:
train.label.value_counts()

label
1    70048
0    34477
Name: count, dtype: int64

In [79]:
print('Dataset information:')
print(f'Training data: {train.shape}')
print(f'Validation data: {valid.shape}')

Dataset information:
Training data: (104525, 2)
Validation data: (3233, 2)


In [80]:
valid

,text,label
0,your empty tin headed brain can concoct good s...,1
1,2007 congress repeated this promise guj hoping...,1
2,election tracker live chidambaram defends nyay...,1
3,modi steps clean governance fail our duty elec...,1
4,the best leader the worldonly narendra bhai modi,1
...,...,...
3228,some people always look the darker side every ...,0
3229,policy initiative party acts action lazy leade...,0
3230,abhinandan was pilot project modi shouted abt ...,0
3231,jai hind modi very nice thought,1


In [81]:
valid.label.value_counts()

label
1    2201
0    1032
Name: count, dtype: int64

In [82]:
from datasets import Dataset
# Combine Multiple Datasets 
train_ds = Dataset.from_pandas(train)
valid_ds = Dataset.from_pandas(valid)
# # Combine Multiple Datasets 
# train_ds = Dataset.from_pandas(train).remove_columns('__index_level_0__')
# valid_ds = Dataset.from_pandas(valid).remove_columns('__index_level_0__')


In [83]:
from transformers import AutoTokenizer

tokenizer = AutoTokenizer.from_pretrained("distilbert-base-uncased")

In [84]:
def preprocess_function(examples):
    return tokenizer(examples["text"], truncation=True)

In [85]:
tokenized_train = train_ds.map(preprocess_function)
tokenized_valid = valid_ds.map(preprocess_function)

  0%|          | 0/104525 [00:00<?, ?ex/s]

  0%|          | 0/3233 [00:00<?, ?ex/s]

In [86]:
from transformers import DataCollatorWithPadding

data_collator = DataCollatorWithPadding(tokenizer=tokenizer)

In [87]:
pip install evaluate

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
Note: you may need to restart the kernel to use updated packages.


In [88]:
import evaluate

accuracy = evaluate.load("accuracy")

In [89]:
import numpy as np


def compute_metrics(eval_pred):
    predictions, labels = eval_pred
    predictions = np.argmax(predictions, axis=1)
    acc = accuracy.compute(predictions=predictions, references=labels)
    print(acc)
    return acc

In [90]:
id2label = {0: "negative", 1: "postive"}
label2id = {"negative": 0, "postive": 1}

In [91]:
from transformers import AutoModelForSequenceClassification, TrainingArguments, Trainer

model = AutoModelForSequenceClassification.from_pretrained(
    "distilbert-base-uncased", num_labels=2, id2label=id2label, label2id=label2id
)

Some weights of DistilBertForSequenceClassification were not initialized from the model checkpoint at distilbert-base-uncased and are newly initialized: ['pre_classifier.bias', 'pre_classifier.weight', 'classifier.weight', 'classifier.bias']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [92]:
import os
os.environ['WANDB_DISABLED'] = 'true'

In [93]:
training_args = TrainingArguments(
    output_dir="./",
    learning_rate=2e-5,
    per_device_train_batch_size=48,
    per_device_eval_batch_size=48,
    num_train_epochs=2,
    weight_decay=0.01,
    evaluation_strategy="steps",
    save_strategy="steps",
    eval_steps= 502,
    save_steps = 502,
    load_best_model_at_end=True,
    push_to_hub=False,
    report_to='none',
    metric_for_best_model = 'accuracy',
    save_total_limit=2,
    overwrite_output_dir=True,
)

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_train,
    eval_dataset=tokenized_valid,
    tokenizer=tokenizer,
    data_collator=data_collator,
    compute_metrics=compute_metrics,
)

trainer.train()

#   5db1d7e350565d7bff6d94eef3e83bee4d3cd460

You're using a DistilBertTokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is faster than using a method to encode the text followed by a call to the `pad` method to get a padded encoding.


Step,Training Loss,Validation Loss,Accuracy
502,0.367300,0.188805,0.926075
1004,0.187800,0.138418,0.949582
1506,0.146600,0.110723,0.960099
2008,0.123900,0.085552,0.972471
2510,0.091800,0.076881,0.974018
3012,0.077600,0.074475,0.972781
3514,0.072600,0.069534,0.976492
4016,0.072000,0.065274,0.977111


{'accuracy': 0.9260748530776368}
{'accuracy': 0.949582431178472}
{'accuracy': 0.960098979276214}
{'accuracy': 0.9724713888029693}
{'accuracy': 0.9740179399938138}
{'accuracy': 0.9727806990411383}
{'accuracy': 0.9764924218991649}
{'accuracy': 0.9771110423755026}


TrainOutput(global_step=4356, training_loss=0.13601346668352, metrics={'train_runtime': 684.1141, 'train_samples_per_second': 305.578, 'train_steps_per_second': 6.367, 'total_flos': 3182290707902664.0, 'train_loss': 0.13601346668352, 'epoch': 2.0})

In [95]:
from transformers import pipeline

# load from previously saved model
classifier = pipeline("text-classification", model="/kaggle/working/checkpoint-4016")

In [97]:
new_data = "I hate any one who can hurt you ".split('.')

preds = classifier(new_data, return_all_scores=True)
preds

[[{'label': 'negative', 'score': 0.9988413453102112},
  {'label': 'postive', 'score': 0.0011587075423449278}]]